In [7]:
import pandas as pd
import plotly.express as px
# from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer
import torch
import hdbscan
import umap
from tqdm import tqdm

# Data Preprocessing and Wrangling

In [8]:
# Load data
df_keywords = pd.read_json('/mnt/hdd01/patentsview/Patentsview - Cleantech Patents/Cleantech Concepts/df_cpc_subclass_keywords_yake.json')

In [10]:
# Explode 'keywords_yake_desc_5000' column
df_keywords_list = df_keywords.explode('keywords_yake_desc_5000')
df_keywords_list = df_keywords_list.reset_index(drop=True)

In [11]:
# Delete column keywords_yake and keywords_yake_desc
df_keywords_list = df_keywords_list.drop(['keywords_yake', 'keywords_yake_desc'], axis=1)

# Embedding

In [13]:
# Ensure a GPU is available and select it for processing
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f'Using GPU: {torch.cuda.get_device_name()}')
else:
    device = torch.device("cpu")
    print('No GPU available, using CPU.')

Using GPU: NVIDIA RTX A4500


In [22]:
# Load the Sentence Transformer model, try 'AI-Growth-Lab/PatentSBERTa' or 'anferico/bert-for-patents' or 'climatebert/distilroberta-base-climate-f'
model = SentenceTransformer('climatebert/distilroberta-base-climate-f', device=device)

No sentence-transformers model found with name /home/thiesen/.cache/torch/sentence_transformers/climatebert_distilroberta-base-climate-f. Creating a new one with MEAN pooling.
Some weights of RobertaModel were not initialized from the model checkpoint at /home/thiesen/.cache/torch/sentence_transformers/climatebert_distilroberta-base-climate-f and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
# Convert the column of lists of keywords to a list of strings
keywords_list = df_keywords_list['keywords_yake_desc_5000'].tolist()

# Compute embeddings (the model will automatically use the GPU if available)
keywords_embeddings = model.encode(keywords_list, convert_to_tensor=True)

# Convert embeddings tensor to a list of lists and add to dataframe
df_keywords_list['keywords_embeddings_climatebert'] = keywords_embeddings.cpu().numpy().tolist()

In [26]:
df_keywords_list.to_json('/mnt/hdd01/patentsview/Patentsview - Cleantech Patents/Cleantech Concepts/df_cpc_subclass_keywords_yake_embeddings.json', orient='records')

# Dimensionality Reduction with UMAP

In [ ]:
# Perform dimensionality reduction with UMAP
reducer = umap.UMAP(random_state=42)
embeddings = df_keywords_list['keywords_embeddings_patentsberta'].tolist()
umap_embeddings = reducer.fit_transform(embeddings)

# Add UMAP embeddings to dataframe
df_keywords_list['keywords_embeddings_patentsberta_umap'] = umap_embeddings.tolist()

# Clustering with HDBSCAN

In [ ]:
# Perform clustering with HDBSCAN
clusterer = hdbscan.HDBSCAN(min_cluster_size=10, metric='euclidean', cluster_selection_method='eom')
clusterer.fit(umap_embeddings)

# Add HDBSCAN clusters to dataframe
df_keywords_list['keywords_embeddings_patentsberta_hdbscan'] = clusterer.labels_.tolist()